In [37]:
import time
from collections import Counter
from itertools import count
class HMM:

    reference = {}
    tag_count = {}
    word_tag = []
    word_tag_em = {}
    tags = ["B-negative","B-neutral","B-positive","I-negative","I-neutral","I-positive","O"]
    new_word_tag = ""
    
    def __init__(self,train_x,train_y):
        self.train_x = train_x # train_x is list of words in training data
        self.train_y = train_y # train_y is list of tags in training data related to x
        self.tag_count = Counter(train_y)
        self.word_tag = Counter(list(zip(train_x,train_y)))
        self.new_word_tag = self.new_word_emission()
        print(self.new_word_tag)
        #print(self.tag_count)
        #print(self.word_tag)

    #part2
    #counting number of occurence of y_i
    def count_y(self,y_i):                
        return self.tag_count[y_i]
    
    
    #2a count(y pair x)/ count(y)
    def est_emission(self,x_i,y_i):
        return self.word_tag[(x_i,y_i)]
    
    
    #2b
    def impr_est_emission(self,x_i,y_i):           
        return self.word_tag[(x_i,y_i)]/(self.count_y(y_i)+1)
        
    def new_word_emission(self):
        score=[]
        for tag in self.tags:
            score.append(1/(self.count_y(tag)+1))
        return self.tags[score.index(max(score))]
        
    def train(self):
        for i in self.word_tag:
            self.word_tag_em[i]=self.impr_est_emission(i[0],i[1])
        return self.word_tag_em
            
    #2c
    def sentiment_analysis(self, test_x):
        print("predicting...")
        y_star=[]
        
        #for each word in new training data eg. hello
        for word in test_x:
            if word not in self.train_x:
                y_star.append(self.new_word_tag)
            else:
                em_list = []
                
                #for each tag in tags eg. B+, B-, Bn 
                for tag in self.tags:
                    if (word,tag) not in self.word_tag_em:
                        em_list.append(0)
                    else:
                        em_list.append(self.word_tag_em[(word,tag)])
                        
                index_of_max = em_list.index(max(em_list))
                predicted_tag = self.tags[index_of_max]
                if predicted_tag[0] == "I":
                    if y_star != [] and y_star[-1]=="O" or y_star==[]:
                        predicted_tag = "B"+predicted_tag[1:]
                    
                y_star.append(predicted_tag)
                    
                
        print("done predicting!")
        return y_star



# format: [[words], [tags]]
def read_train(file_name):
    in_file = open(file_name,'r',encoding='utf8')
    l = []
    words = []
    tags = []
    for line in in_file:
        x = line.strip().split()
        if x != []:
            words.append(x[0])
            tags.append(x[1].rstrip('\n'))
    l.append(words)
    l.append(tags)
    return l
    
#reading and writing to files 
# format:[words]
def read_dev_in(file_name):
    in_file = open(file_name,'r',encoding='utf8')
    l = []
    for line in in_file:
        l.append(line.strip())
    return l


train_data = read_train("EN/train")
test_data = read_dev_in("EN/dev.in")
part2_HMM = HMM(train_data[0],train_data[1])

#print("ANSWER: "+str(part2_HMM.count_y('O')))
#print("ANSWER: "+str(part2_HMM.est_emission('in','O')))
starttime = time.time()
part2_HMM.train()
test_tags = part2_HMM.sentiment_analysis(test_data)
print("number of words = "+str(len(test_data)))
print("number of tags = "+str(len(test_tags)))

elapsed = time.time()-starttime

print ("time taken = "+str(elapsed)+"s")
#result = part2_HMM.sentiment_analysis(test_data)
#print(result)


I-negative
predicting...
done predicting!
number of words = 8466
number of tags = 8466
time taken = 1.4250094890594482s
